This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

wLEuM_BQzm7U7RLX8Kzx


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params={'api_key': API_KEY})

In [7]:
dataset = r.json()['dataset_data']
dataset.keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(dataset['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [9]:
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-25'}
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=params)

In [10]:
dataset = r.json()['dataset_data']
print(dataset['column_names'])
print(dataset['start_date'])
print(dataset['end_date'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
2017-01-01
2017-12-25


2. Convert the returned JSON object into a Python dictionary.

In [11]:
data_list = dataset['data']
column_names = dataset['column_names']

data = []
for row in data_list:
  row_dict = {}
  for index, key in enumerate(column_names):
    row_dict[key] = row[index]
  # excludes some rows with missing data
  if(row_dict['Open']):
    data.append(row_dict)

# (is that helpful?)

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [12]:
max_opening_price = max(map(lambda row: row['Open'], data))
print('highest opening price: ${:2.2f}'.format(max_opening_price))

min_opening_price = min(map(lambda row: row['Open'], data))
print('lowest opening price: ${:2.2f}'.format(min_opening_price))

highest opening price: $53.11
lowest opening price: $34.00


4. What was the largest change in any one day(based on High and Low price)?

In [13]:
max_day_change = max(map(lambda row: row['High'] - row['Low'], data))
print('largest change in one day: ${:2.2f}'.format(max_day_change))

largest change in one day: $2.81


5. What was the largest change between any two days(based on Closing Price)?

In [14]:
max_between_day_change = 0
for (index, row) in enumerate(data):
    if index > 0:
      diff = abs(row['Close'] - data[index-1]['Close'])
      max_between_day_change = max(diff, max_between_day_change)

print('largest change between two consecutive days: ${:2.2f}'.format(max_between_day_change))

largest change between two consecutive days: $2.56


6. What was the average daily trading volume during this year?

In [15]:
N = len(data)
avg_traded_volume = sum(map(lambda row: row['Traded Volume'], data)) / N

print('average daily trading volume: ${:2.2f}'.format(avg_traded_volume))

average daily trading volume: $89681.82


7. (Optional) What was the median trading volume during this year?

In [21]:
def median(arr):
  sorted_arr = sorted(arr)
  if(len(arr) % 2 == 0):
    mid_index = len(arr) // 2
    return (sorted_arr[mid_index] + sorted_arr[mid_index - 1]) / 2
  else:
    mid_index = (len(arr) - 1) // 2
    return sorted_arr[mid_index]

assert median([0,1,2]) == 1
assert median([2,1,3]) == 2
assert median([0,1,3,4]) == 2

In [22]:
median_traded_volume = median(list(map(lambda row: row['Traded Volume'], data)))
print('median daily trading volume: ${:2.2f}'.format(median_traded_volume))

median daily trading volume: $76286.00
